In [1]:
import h5py
from glob import glob
import sys, scipy
from scipy.stats import chi2
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.utils import shuffle
from prettytable import PrettyTable

import gpustat
gpustat.print_gpustat()
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE" # Some versions of HDF5 require this
os.environ['CUDA_VISIBLE_DEVICES']='2' # This is to choose which GPU to use

import h5py
import numpy as np

from torch.utils.data import Dataset, DataLoader # 
from torch.autograd import Variable

import torch
import torch.nn as nn
import torch.nn.functional as F

gpu-4-culture-plate-sm  Fri Jul  5 09:18:34 2019
[0] GeForce GTX 1080 | 77'C,  93 % |  1767 /  8114 MB | emoreno(1757M)
[1] GeForce GTX 1080 | 27'C,   0 % |     0 /  8114 MB |
[2] GeForce GTX 1080 | 34'C,   0 % |     0 /  8114 MB |
[3] GeForce GTX 1080 | 24'C,   0 % |     0 /  8114 MB |
[4] GeForce GTX 1080 | 26'C,   0 % |     0 /  8114 MB |
[5] GeForce GTX 1080 | 33'C,   0 % |     0 /  8114 MB |
[6] GeForce GTX 1080 | 28'C,   0 % |  5791 /  8114 MB | thong(5781M)
[7] GeForce GTX 1080 | 70'C,  92 % |  5797 /  8114 MB | thong(5787M)


In [2]:
INPUT_LENGTH = 20 # number of particles to consider in 1 event
INPUT_FEATURE = 3 # number of features. Only use pt, eta, phi now
BATCH_SIZE = 10
PT_SCALE = 10
MAX_EPOCH = 20
# Maurizio's Delphes data
# this is the location on the caltech machine. Copied from /eos/project/d/dshep/TOPCLASS/L1jetLepData/
base_dir = '/bigdata/shared/L1AnomalyDetection/qcd_lepFilter_13TeV/' 
bsm_dir = '/bigdata/shared/L1AnomalyDetection/Ato4l_lepFilter_13TeV/'
# particle is ordered: MET + 10 e + 10 mu + 20 jet
# We will take 4 ele + 4 muon + 12 jets = 20 objects in total

PARTICLE_TO_USE = np.asarray([False]*41)
PARTICLE_TO_USE[1:5] = True # 4 ele
PARTICLE_TO_USE[11:15] = True # 4 muon
PARTICLE_TO_USE[21:33] = True # 12 jets

In [3]:
# class SimpleEventSequence(Dataset):
#     def __init__(self, data_x, data_y):
#         self.len = data_x.shape[0]
#         self.data_x = torch.from_numpy(data_x).float()
#         self.data_y = torch.from_numpy(data_y)
        
#     def __len__(self):
#         return self.len

#     def __getitem__(self, idx):
#         return (self.data_x[idx], self.data_y[idx])

class EventSequence(Dataset):
    def check_data(self, file_names):
        num_data = 0
        thresholds = [0]
        for in_file_name in file_names:
            h5_file = h5py.File( in_file_name, 'r' )
            X = h5_file[self.feature_name]
            if hasattr(X, 'keys'):
                num_data += len(X[X.keys()[0]])
                thresholds.append(num_data)
            else:
                num_data += len(X)
                thresholds.append(num_data)
            h5_file.close()
        return (num_data, thresholds)

    def __init__(self, dir_name, feature_name = 'Particles', sequence_length=50, verbose=False):
        self.feature_name = feature_name
        self.file_names = glob(dir_name+'/*.h5')
        self.num_data, self.thresholds = self.check_data(self.file_names)
        self.sequence_length = sequence_length
        self.file_index = 0
        self.h5_file = h5py.File(self.file_names[self.file_index],'r')
        self.get_data()
        self.verbose=verbose
        
    def get_data(self):
        self.X = np.array(self.h5_file.get(self.feature_name))[:,PARTICLE_TO_USE,:] # Skip the class feature 

    def is_numpy_array(self, data):
        return isinstance(data, np.ndarray)

    def get_num_samples(self, data):
        """Input: dataset consisting of a numpy array or list of numpy arrays.
            Output: number of samples in the dataset"""
        if self.is_numpy_array(data):
            return len(data)
        else:
            return len(data[0])

    def get_index(self, idx):
        """Translate the global index (idx) into local indexes,
        including file index and event index of that file"""
        file_index = next(i for i,v in enumerate(self.thresholds) if v > idx)
        file_index -= 1
        event_index = idx - self.thresholds[file_index]
        return file_index, event_index

    def get_thresholds(self):
        return self.thresholds

    def __len__(self):
        return self.num_data

    def __getitem__(self, idx):
        file_index, event_index = self.get_index(idx)
        
        if file_index != self.file_index:
            self.h5_file.close()
            self.file_index = file_index
            if self.verbose: 
                print("Opening new file {}".format(self.file_names[self.file_index]))
            self.h5_file = h5py.File(self.file_names[self.file_index],'r')
            self.get_data()
            self.X = shuffle(self.X)
        #return [self.X[event_index], np.argmax(self.Y[event_index])]
        X_ = self.X[event_index]
        if not np.any(X_): # empty array, go to the next event
            #print("Empty event. Skipping.")
            return self.__getitem__(idx+1)
        #print("Getting event.")
        # Scale down pT
        X = X_
        X[:,0] = X_[:,0]/PT_SCALE
        
        #X = X.view((INPUT_LENGTH*INPUT_FEATURE,) # flatten
        X = X[:,:3].flatten()
        return X

    


In [4]:
class Autoencoder(nn.Module):
    def __init__(self, n_features):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(n_features, int(n_features*2/3)),
            nn.ReLU(True),
            nn.Linear(int(n_features*2/3), int(n_features/3)),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.Linear(int(n_features/3), int(n_features*2/3)),
            nn.ReLU(True),
            nn.Linear(int(n_features*2/3), n_features)
            )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [5]:
# This will take a bit long, because it needs to open through all 
# files to get the number of events in each files
train_loader = DataLoader(EventSequence(dir_name=base_dir,
                                    feature_name ='Particles', sequence_length=INPUT_LENGTH, verbose=False), 
                                    batch_size = BATCH_SIZE, shuffle=False,num_workers=3)



In [6]:
FLAT_FEATURES = INPUT_LENGTH*INPUT_FEATURE
model = Autoencoder(FLAT_FEATURES).cuda()
print(model)
trainablePars = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('\nTrainable parameters:', trainablePars)


Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=60, out_features=40, bias=True)
    (1): ReLU(inplace)
    (2): Linear(in_features=40, out_features=20, bias=True)
    (3): ReLU(inplace)
  )
  (decoder): Sequential(
    (0): Linear(in_features=20, out_features=40, bias=True)
    (1): ReLU(inplace)
    (2): Linear(in_features=40, out_features=60, bias=True)
  )
)

Trainable parameters: 6560


In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = nn.L1Loss()

from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, 
                              mode='min',
                              factor=0.3,
                              patience=3,
                              verbose=1,
                              threshold=1e-4,
                              cooldown=2,
                              min_lr=1e-7
                             )

train_loss = []
loss_history = {'train': [], 'val': []}
for epoch in range(MAX_EPOCH):
    batch_loss = []
    for batch_idx, local_x in enumerate(train_loader):
        local_x = Variable(local_x).float().cuda() 
        # I use an old version of Pytorch. For version > 0.4, feel free to remove Variable wrapper
        x_prime = model(local_x)
        loss = criterion(x_prime, local_x)
        batch_loss.append(loss.data[0])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, MAX_EPOCH, loss.data[0]))

Epoch [1/20], loss:0.0493
Epoch [2/20], loss:0.0504
Epoch [3/20], loss:0.0503
Epoch [4/20], loss:0.0507
Epoch [5/20], loss:0.0509
Epoch [6/20], loss:0.0499
Epoch [7/20], loss:0.0505
Epoch [8/20], loss:0.0499
Epoch [9/20], loss:0.0496
Epoch [10/20], loss:0.0483
Epoch [11/20], loss:0.0485
Epoch [12/20], loss:0.0484
Epoch [13/20], loss:0.0482
Epoch [14/20], loss:0.0481
Epoch [15/20], loss:0.0491
Epoch [16/20], loss:0.0493
Epoch [17/20], loss:0.0492
Epoch [18/20], loss:0.0493
Epoch [19/20], loss:0.0508
Epoch [20/20], loss:0.0489
